# Amazon Redshift - Load TSV Data Into Redshift

TODO: Describe scenario

<img src="img/redshift_load_tsv.png" width="55%" align="left">

In [ ]:
import boto3
import sagemaker

# Get region 
session = boto3.session.Session()
region_name = session.region_name

# Get SageMaker session & default S3 bucket
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

# Set S3 prefixes
tsv_prefix = 'amazon-reviews-pds/tsv'

# Set S3 destination paths
s3_destination_path_tsv = 's3://{}/{}'.format(bucket, tsv_prefix)


## Setup Redshift Connection Via SQLAlchemy
https://pypi.org/project/SQLAlchemy/

In [ ]:
!pip install -q SQLAlchemy==1.3.13

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pandas as pd


# TODO:  Change all variable names to lower_case - not UPPER CASE.  
#        (and not SetPath...change to set_path)

In [ ]:
# TODO: get Endpoint name programatically
redshift_endpoint = '<endpoint-name>'
redshift_port = '5439'

SCHEMA = 'public'
table_name_tsv = 'amazon_reviews_tsv'

DB_NAME = 'dsoaws'

MASTER_USER_NAME = 'dsoaws'
MASTER_USER_PW = '<password>'


In [ ]:
# Connect to Redshift database engine
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(MASTER_USER_NAME, MASTER_USER_PW, redshift_endpoint, redshift_port, DB_NAME))


In [ ]:
# Configure Session
session = sessionmaker()
session.configure(bind=engine)
s = session()
SetPath = "SET search_path TO %s" % SCHEMA
s.execute(SetPath)

### Create TSV Table In Redshift

In [ ]:
statement = """DROP TABLE IF EXISTS {}""".format(table_name_tsv)

s = session()
s.execute(statement)

In [ ]:
statement = """CREATE TABLE {}( 
         marketplace varchar(2),
         customer_id varchar(8),
         review_id varchar(14),
         product_id varchar(10),
         product_parent varchar(9),
         product_title varchar(400),
         product_category varchar(24),
         star_rating int,
         helpful_votes int,
         total_votes int,
         vine varchar(1),
         verified_purchase varchar(1),
         review_headline varchar(128),
         review_body varchar(65535),
         review_date varchar(10)
)""".format(table_name_tsv)


In [ ]:
s.execute(statement)
s.commit()

## Load TSV Data From S3 Into Redshift

Note: The statement below runs for approx. 30min. 

In [ ]:
statement = """COPY {} (marketplace, customer_id, review_id, product_id, 
                product_parent, product_title, product_category, star_rating, helpful_votes, total_votes, 
                vine, verified_purchase, review_headline, review_body, review_date) 
            FROM \'{}/\'
            IAM_ROLE \'{}\'
            IGNOREHEADER 1 DELIMITER '\\t' 
            GZIP TRUNCATECOLUMNS;""".format(table_name_tsv, s3_destination_path_tsv, IAM_ROLE)
print(statement)

In [ ]:
s = session()
s.execute(statement)
s.commit()

### Run a sample query

In [ ]:
df = pd.read_sql_query("""SELECT product_category,
                            COUNT(star_rating) AS count_star_rating
                        FROM {}
                        GROUP BY product_category
                        ORDER BY count_star_rating DESC""".format(table_name_tsv), engine)

In [ ]:
df.head(5)